In [6]:
!pip install scikit-learn

In [7]:
!pip install nltk


In [8]:
!pip install pandas

In [35]:
import pandas as pd
import nltk
from nltk.corpus import twitter_samples
import random
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from nltk.classify import NaiveBayesClassifier
from gensim.models import Word2Vec
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
nltk.download('twitter_samples')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')
import nltk.classify.util


df=pd.read_csv('youtoxic_english_1000(2).csv',usecols=[2,15])
# podzial na texty i ich kasyfikacje
i=0
banedComments=[]
notBanedComments=[]
textsArray=df.drop("IsBaned",axis=1).to_numpy()
IsBanedArray=df["IsBaned"].to_numpy()

# funkcja do tworzenia arrayow z labelem true i zlabelem false
def add_to_array(text,index):
  # print(textsArray[index][0])
  if(text==True):
    banedComments.append(textsArray[index][0])
  else:
    notBanedComments.append(textsArray[index][0])
# tworzenie arrayow
for i in range(len(IsBanedArray)):
  add_to_array(IsBanedArray[i],i)

# robiwnie labelow i ich tasowanie
comments=banedComments+notBanedComments
labels = ['True'] * len(banedComments) + ['False'] * len(notBanedComments)
# comments
# Shuffle the dataset
combined = list(zip(comments, labels))
random.shuffle(combined)
comments, labels = zip(*combined)
# array to text i tokenizacja
sample_text = ','.join(comments)
tokens = word_tokenize(sample_text)

# usuwanie tokenów
stop_words = set(stopwords.words('english'))
def remove_stopwords(tokens):
    return [word for word in tokens if word.lower() not in stop_words]
filtered_tokens = remove_stopwords(tokens)


# from gensim.models import Word2Vec
# import numpy as np

# Train Word2Vec model
tokenized_comments = [word_tokenize(comment.lower()) for comment in comments]
word2vec_model = Word2Vec(sentences=tokenized_comments, vector_size=100, window=5, min_count=2, workers=4)

# Compute average Word2Vec vector for each comment
def document_vector(words, model):
    words = [word for word in words if word in model.wv]
    if not words:
        return np.zeros(model.vector_size)
    return np.mean(model.wv[words], axis=0)

X = np.array([document_vector(word_tokenize(comment.lower()), word2vec_model) for comment in comments])

# Train-test split and classifier
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.3, random_state=42)
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# //////////////////////////////////////////
# function to get precision recall and accuracy
def classiffy_train_set(text):
    prediction = classifier.predict(text)
    return prediction
# ocena modela
number=300
TP=0
TN=0
FP=0
FN=0
predictions=classiffy_train_set(X_test)
# print(IsBanedArray[0])
for i in range(number):
  if(str(y_test[i])=="True"):
    if(str(predictions[i])==str(y_test[i])):
      TP=TP+1
    else:
      FN=FN+1
  else:
    if(str(predictions[i])==str(y_test[i])):
      TN=TN+1
    else:
      FP=FP+1
print ("true positive=  "+str(TP))
print ("true negative=  "+str(TN))
print ("false positive=  "+str(FP))
print ("false negative=  "+str(FN))
precision=TP/(TP+FP)
recall=TP/(TP+FN)
accuracy=(TP+TN)/number
print("precision= "+str(precision))
print("recall= "+str(recall))
print("accuracy= "+str(accuracy))


# definicja nowego textu(nie w training set)
def classiffy_text(text):
    # Tokenize and preprocess the text
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]

    # Compute the Word2Vec vector for the input text
    text_vector = document_vector(tokens, word2vec_model)

    # Reshape the vector for prediction (1 sample, n features)
    text_vector = text_vector.reshape(1, -1)

    # Make the prediction
    prediction = classifier.predict(text_vector)
    # print(prediction)
    return prediction[0]



[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Accuracy: 53.00%
true positive=  54
true negative=  105
false positive=  54
false negative=  87
precision= 0.5
recall= 0.3829787234042553
accuracy= 0.53


In [36]:
print(classiffy_text("This comment seems offensive and should be banned"))


True
